# csv hdfs 읽고 쓰기  
spark을 이용해 rdd로 샘플 데이터를 생성하고, 이를 hdfs 에 csv 파일로 기록한다.  
기록에 성공하면, 반대로 csv file을 spark에서 data로 읽어 들여 사용해 본다.  

In [1]:
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [2]:
# 스파크 생성 
spark = SparkSession.builder.master('yarn').appName('Data_Sources')\
    .config("spark.sql.warehouse.dir", '/user/root/inven')\
    .getOrCreate()

sc = spark.sparkContext
sc

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


<SparkContext master=yarn appName=Data_Sources>

In [10]:
# 샘플 데이터 생성 
def create_setops():
    setop_count = 10000
    setop_name = ['ST_A', 'ST_B', 'ST_C', 'ST_D', 'ST_E']
    setops = []
    for s in setop_name:
        for i in range(0, int(setop_count/len(setop_name))):
            setops.append([f'{s}_{i:03d}', setop_count])
            
    print(setops[-10:])
    return setops
# 샘플 데이터 형식 정의. 읽기/쓰기 편의 제공. 
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType
    columns = [
        StructField("setop", StringType())
        , StructField("remains", LongType())
    ]
    sample_schema = StructType(columns)
    return sample_schema

In [11]:
# 샘플 데이터 생성 및 확인 
sample_data = create_setops()
sample_schema = define_schema()
rdd = spark.sparkContext.parallelize(sample_data)
df = spark.createDataFrame(rdd, sample_schema)
df.show(10)


[['ST_E_1990', 10000], ['ST_E_1991', 10000], ['ST_E_1992', 10000], ['ST_E_1993', 10000], ['ST_E_1994', 10000], ['ST_E_1995', 10000], ['ST_E_1996', 10000], ['ST_E_1997', 10000], ['ST_E_1998', 10000], ['ST_E_1999', 10000]]
+--------+-------+
|   setop|remains|
+--------+-------+
|ST_A_000|  10000|
|ST_A_001|  10000|
|ST_A_002|  10000|
|ST_A_003|  10000|
|ST_A_004|  10000|
|ST_A_005|  10000|
|ST_A_006|  10000|
|ST_A_007|  10000|
|ST_A_008|  10000|
|ST_A_009|  10000|
+--------+-------+
only showing top 10 rows



In [12]:
# HDFS 에 /user/root/inven/setop 폴더와 파일이 생성 됨.  
df.write.save(path='inven/setop', format='csv', mode='append', sep=',')

In [14]:
%%time 
# 기록한 파일 다시 읽어 들이기 
# 저장 결과 확인하기 
lines = spark.read.format("csv").option('path', "inven/setop").load()
data_count = lines.count()
print(f'DATA Count : {data_count:,}')
lines.show(5)

DATA Count : 10,000
+--------+-----+
|     _c0|  _c1|
+--------+-----+
|ST_A_000|10000|
|ST_A_001|10000|
|ST_A_002|10000|
|ST_A_003|10000|
|ST_A_004|10000|
+--------+-----+
only showing top 5 rows

CPU times: user 3.95 ms, sys: 812 µs, total: 4.76 ms
Wall time: 842 ms


In [15]:
spark.stop()